<a href="https://colab.research.google.com/github/beyzaturku/Weather_Detection_Using_CNN/blob/main/CNN_LSTM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Google Colab ve Drive Bağlantısı


In [1]:
from google.colab import drive
import os
import glob
drive.mount('/content/drive') #google drive ı bağlamak için

Mounted at /content/drive


# Veri Ön İşleme


*   Görüntülerin yeniden boyutlandırılması
*   Verilerin eğitim ve test olarak ayrılması
*   Verilerin normalleştirilmesi



In [2]:
import numpy as np
import pandas as pd
import tensorflow as tf
import os
import cv2
import matplotlib.pyplot as plt
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, LSTM, Dense, TimeDistributed, Dropout
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split
from google.colab import drive

In [28]:
# Veri seti yolu
image_folder = "/content/drive/MyDrive/Weather_Detection/Multi_Class_Weather_Dataset"
img_size = (64,64) #Görüntünün yeniden boyutlandırıldığı boyut


subfolders = ['Cloudy', 'Rain', 'Shine', 'Sunrise']
label_map = {subfolder: idx for idx, subfolder in enumerate(subfolders)}

# Görüntü ve etiketlerin listelenmesi için:
images = []
labels = []

#Veri setini yüklemek için:
def load_images_from_folder(folder):
    images = []
    labels = []
    class_names = os.listdir(folder)
    class_map = {class_name: idx for idx, class_name in enumerate(class_names)}

    for class_name in class_names:
        class_folder = os.path.join(folder, class_name)
        if os.path.isdir(class_folder):
            for img_name in os.listdir(class_folder):
                img_path = os.path.join(class_folder, img_name)
                img = cv2.imread(img_path)
                if img is not None:
                    img = cv2.resize(img, img_size)
                    images.append(img)
                    labels.append(class_map[class_name])

    return np.array(images), np.array(labels), class_map

  # Veriyi yükleyelim
images, labels, class_map = load_images_from_folder(image_folder)
images = images / 255.0  # Normalizasyon

print("Veri yükleniyor...")
X, y, class_map = load_images_from_folder(image_folder)
print("Toplam görüntü sayısı:", X.shape[0])

Veri yükleniyor...
Toplam görüntü sayısı: 1123


In [36]:
seq_length = 1 #LSTM için ardışık görüntü sayısı

# Görüntüleri zaman serisine çevirme
def create_sequences(images, labels, seq_length):
    X_seq, y_seq = [], []
    for i in range(len(images) - seq_length):
        X_seq.append(images[i:i + seq_length])  # 5 görüntü
        y_seq.append(labels[i + seq_length - 1])  # Son görüntünün etiketi
    return np.array(X_seq), np.array(y_seq)

X_seq, y_seq = create_sequences(X, y, seq_length)
print("Yeni giriş şekli:", X_seq.shape)  # (örnek_sayısı, 5, 64, 64, 3)

Yeni giriş şekli: (1122, 1, 64, 64, 3)


In [37]:
# Veri setinin eğitim ve test olarak bölünmesi
split = int(0.8 * len(X_seq))
X_train, y_train = X_seq[:split], y_seq[:split]
X_test, y_test = X_seq[split:], y_seq[split:]

In [38]:
#CNN-LSTM Model Mimarisi

from tensorflow.keras import Input

def build_model():
    model = Sequential([
        Input(shape=(seq_length, 64, 64, 3)),
        TimeDistributed(Conv2D(32, (3, 3), activation='relu')),
        TimeDistributed(MaxPooling2D((2, 2))),
        TimeDistributed(Conv2D(64, (3, 3), activation='relu')),
        TimeDistributed(MaxPooling2D((2, 2))),
        TimeDistributed(Flatten()),
        LSTM(64, return_sequences=False),
        Dense(64, activation='relu'),
        Dropout(0.5),
        Dense(len(class_map), activation='softmax')
    ])

    model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    return model

print("Model oluşturuluyor...")
model = build_model()

Model oluşturuluyor...


In [23]:
#model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [39]:
# Modeli eğitme
print("Model eğitiliyor...")
history = model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=20, batch_size=32)

Model eğitiliyor...
Epoch 1/20
29/29 ━━━━━━━━━━━━━━━━━━━━ 4s 33ms/step - accuracy: 0.2635 - loss: 1.3834 - val_accuracy: 0.0000e+00 - val_loss: 2.3866
Epoch 2/20
29/29 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.3669 - loss: 1.3140 - val_accuracy: 0.0000e+00 - val_loss: 2.5374
Epoch 3/20
29/29 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.3506 - loss: 1.3253 - val_accuracy: 0.0000e+00 - val_loss: 2.3807
Epoch 4/20
29/29 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.3377 - loss: 1.3117 - val_accuracy: 0.0000e+00 - val_loss: 2.3460
Epoch 5/20
29/29 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.3585 - loss: 1.2883 - val_accuracy: 0.0000e+00 - val_loss: 2.4442
Epoch 6/20
29/29 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.3709 - loss: 1.2736 - val_accuracy: 0.0000e+00 - val_loss: 2.4795
Epoch 7/20
29/29 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.3671 - loss: 1.2815 - val_accuracy: 0.0000e+00 - val_loss: 2.4432
Epoch 8/20
29/29 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 

In [40]:
# Test doğruluğu
test_loss, test_acc = model.evaluate(X_test, y_test)
print("Test doğruluğu:", test_acc)

8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.0000e+00 - loss: 2.4327 
Test doğruluğu: 0.0


# **Sonuç:** Veri seti LSTM' e uygun olmadığından dolayı başarısız olduğunu düşünüyorum.